In [1]:
import numpy as np
import pandas as pd
import math
import random
from pprint import pprint

In [2]:
df = pd.read_csv("Titanic.csv")
df["label"] = df.Survived
df = df.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis=1)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,label
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [3]:
median_age = df.Age.median()
mode_embarked = df.Embarked.mode()[0]
df=df.fillna({'Age':median_age,'Embarked':mode_embarked})

In [4]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [5]:
np.unique(df['Age'])

array([ 0.42,  0.67,  0.75,  0.83,  0.92,  1.  ,  2.  ,  3.  ,  4.  ,
        5.  ,  6.  ,  7.  ,  8.  ,  9.  , 10.  , 11.  , 12.  , 13.  ,
       14.  , 14.5 , 15.  , 16.  , 17.  , 18.  , 19.  , 20.  , 20.5 ,
       21.  , 22.  , 23.  , 23.5 , 24.  , 24.5 , 25.  , 26.  , 27.  ,
       28.  , 28.5 , 29.  , 30.  , 30.5 , 31.  , 32.  , 32.5 , 33.  ,
       34.  , 34.5 , 35.  , 36.  , 36.5 , 37.  , 38.  , 39.  , 40.  ,
       40.5 , 41.  , 42.  , 43.  , 44.  , 45.  , 45.5 , 46.  , 47.  ,
       48.  , 49.  , 50.  , 51.  , 52.  , 53.  , 54.  , 55.  , 55.5 ,
       56.  , 57.  , 58.  , 59.  , 60.  , 61.  , 62.  , 63.  , 64.  ,
       65.  , 66.  , 70.  , 70.5 , 71.  , 74.  , 80.  ])

In [12]:
#Data Pure
def check_purity(data):
    label_column=data[:,-1]
    unique_classes=np.unique(label_column)
    if len(unique_classes)==1:
        return True
    else:
        return False


In [13]:

def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification


In [14]:
#Feature type:
def determine_type_of_feature(df):
    features_type=[]
    n_unique_values_treshold = 15
    
    for feature in df.columns:
        if feature != 'label':
            unique_values=df[feature].unique()
            example_value=unique_values[0]
            
            if (isinstance(example_value,str)) or (len(unique_values) <= n_unique_values_treshold ):
                features_type.append('categorical')
            else:
                features_type.append('continuous')
    return features_type

#test:
print(determine_type_of_feature(df))

['categorical', 'categorical', 'continuous', 'categorical', 'categorical', 'continuous', 'categorical']


In [15]:
#Potential Splits:
FEATURE_TYPES=determine_type_of_feature(df)
def get_potential_splits(data): #return dict with key is column index
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):
        values=data[:,column_index]
        unique_values=np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature=="continuous":
            potential_splits[column_index]=[]
            for index in range(len(unique_values)):
                if index != 0:
                    current_value=unique_values[index]
                    previous_value=unique_values[index-1]
                    potential_split=(current_value+previous_value)/2
                    potential_splits[column_index].append(potential_split)
        elif len(unique_values) > 1:
            potential_splits[column_index] = unique_values
    return potential_splits 
#test: 


In [16]:
# get_potential_splits(df.values)

In [17]:
df.values.shape

(891, 8)

In [18]:
#chia du lieu
def split_data(data,split_column,split_value):
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [11]:
split_data(df.values,0,1)

(array([[1, 'female', 38.0, ..., 71.2833, 'C', 1],
        [1, 'female', 35.0, ..., 53.1, 'S', 1],
        [1, 'male', 54.0, ..., 51.8625, 'S', 0],
        ...,
        [1, 'female', 56.0, ..., 83.1583, 'C', 1],
        [1, 'female', 19.0, ..., 30.0, 'S', 1],
        [1, 'male', 26.0, ..., 30.0, 'C', 1]], dtype=object),
 array([[3, 'male', 22.0, ..., 7.25, 'S', 0],
        [3, 'female', 26.0, ..., 7.925, 'S', 1],
        [3, 'male', 35.0, ..., 8.05, 'S', 0],
        ...,
        [2, 'male', 27.0, ..., 13.0, 'S', 0],
        [3, 'female', 28.0, ..., 23.45, 'S', 0],
        [3, 'male', 32.0, ..., 7.75, 'Q', 0]], dtype=object))

### Entropy


In [19]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
#Choose point has lowest value if using that value to split data 
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [21]:
[i for i in get_potential_splits(df.values)]

[0, 1, 2, 3, 4, 5, 6]

In [13]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    else:    
        counter += 1

        # chon ra diem chia tot nhat va chia dl theo diem chia do
        potential_splits = get_potential_splits(data) #shape: (891,8)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # tao cau hoi
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # khoi tao cay
        sub_tree = {question: []}
        
        # recursion
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.# This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [14]:
#classification:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)
    
#Accuracy:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, args=(tree,), axis=1)
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [15]:
train_df, test_df = train_test_split(df, 0.2)
tree = decision_tree_algorithm(train_df, max_depth=3)
accuracy = calculate_accuracy(test_df, tree)

pprint(tree,width=40)
accuracy

{'Sex = male': [0,
                {'Pclass = 3': [{'Fare <= 23.35': [1,
                                                   0]},
                                1]}]}


0.8089887640449438

In [16]:
{'Sex = male': [{'Fare <= 26.26875': [{'Age <= 14.0': [1, 0]}, 0]}, {'Pclass = 3': [{'Fare <= 23.35': [1, 0]}, 1]}]}


{'Sex = male': [{'Fare <= 26.26875': [{'Age <= 14.0': [1, 0]}, 0]},
  {'Pclass = 3': [{'Fare <= 23.35': [1, 0]}, 1]}]}